<a href="https://colab.research.google.com/github/ssosoo/2024_DS60/blob/main/%EB%94%A5%EB%9F%AC%EB%8B%9D/LSTM_%EC%A3%BC%EC%A0%9C%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LSTM을 사용해 각 지문의 주제를 분류하는 코드를 짠다.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

In [3]:
paragraph_dict_list = [
         {'paragraph': 'dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait', 'category': 'food'},
         {'paragraph': 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations', 'category': 'food'},
         {'paragraph': 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table', 'category': 'food'},
         {'paragraph': 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition', 'category': 'food'},
         {'paragraph': 'the biscuits and gravy was too salty two people in my group had the gravy and all thought it was too salty my hubby ordered a side of double egg and it was served on two small plates who serves eggs to one person on separate plates we commented on that when it was delivered and even the server laughed and said she doesnt know why the kitchen does that presentation of food is important and they really missed on this one', 'category': 'food'},
         {'paragraph': 'the garlic fries were a great starter (and a happy hour special) the pancakes looked and tasted great and were a fairly generous portion', 'category': 'food'},
         {'paragraph': 'our meal was excellent i had the pasta ai formaggi which was so rich i didnt dare eat it all although i certainly wanted to excellent flavors with a great texture contrast between the soft pasta and the crisp bread crumbs too much sauce for me but a wonderful dish', 'category': 'food'},
         {'paragraph': 'what i enjoy most about palo alto is so many restaurants have dog-friendly seating outside i had bookmarked italico from when they first opened about a 1.5 years ago and was jonesing for some pasta so time to finally knock that bookmark off', 'category': 'food'},
         {'paragraph': 'the drinks came out fairly quickly a good two to three minutes after the orders were taken i expected my iced tea to taste a bit more sweet but this was straight up green tea with ice in it not to complain of course but i was pleasantly surprised', 'category': 'food'},
         {'paragraph': 'despite the not so good burger the service was so slow the restaurant wasnt even half full and they took very long from the moment we got seated to the time we left it was almost 2 hours we thought that it would be quick since we ordered as soon as we sat down my coworkers did seem to enjoy their beef burgers for those who eat beef however i will not be returning it is too expensive and extremely slow service', 'category': 'food'},

         {'paragraph': 'the four reigning major champions simona halep caroline wozniacki angelique kerber and defending us open champion sloane stephens could make a case for being the quartet most likely to succeed especially as all but stephens has also enjoyed the no1 ranking within the last 14 months as they prepare for their gruelling new york campaigns they currently hold the top four places in the ranks', 'category': 'sports'},
         {'paragraph': 'the briton was seeded nn7 here last year before a slump in form and confidence took her down to no46 after five first-round losses but there have been signs of a turnaround including a victory over a sub-par serena williams in san jose plus wins against jelena ostapenko and victoria azarenka in montreal. konta pulled out of new haven this week with illness but will hope for good things where she first scored wins in a major before her big breakthroughs to the semis in australia and wimbledon', 'category': 'sports'},
         {'paragraph': 'stephens surged her way back from injury in stunning style to win her first major here last year—and ranked just no83 she has since proved what a big time player she is winning the miami title via four fellow major champions then reaching the final at the french open back on north american hard courts she ran to the final in montreal only just edged out by halep she has also avoided many of the big names in her quarter—except for wild card azarenka as a possible in the third round', 'category': 'sports'},
         {'paragraph': 'when it came to england chances in the world cup it would be fair to say that most fans had never been more pessimistic than they were this year after enduring years of truly dismal performances at major tournaments – culminating in the 2014 event where they failed to win any of their three group games and finished in bottom spot those results led to the resignation of manager roy hodgson', 'category': 'sports'},
         {'paragraph': 'the team that eliminated russia – croatia – also improved enormously during the tournament before it began their odds were 33/1 but they played with real flair and star players like luka modric ivan rakitic and ivan perisic showed their quality on the world stage having displayed their potential by winning all three of their group stage games croatia went on to face difficult tests like the semi-final against england', 'category': 'sports'},
         {'paragraph': 'the perseyside outfit finished in fourth place in the premier league table and without a trophy last term after having reached the champions league final before losing to real madrid', 'category': 'sports'},
         {'paragraph': 'liverpool fc will return to premier league action on saturday lunchtime when they travel to leicester city in the top flight as they look to make it four wins in a row in the league', 'category': 'sports'},
         {'paragraph': 'alisson signed for liverpool fc from as roma this summer and the brazilian goalkeeper has helped the reds to keep three clean sheets in their first three premier league games', 'category': 'sports'},
         {'paragraph': 'but the rankings during that run-in to new york hid some very different undercurrents for murray had struggled with a hip injury since the clay swing and had not played a match since losing his quarter-final at wimbledon and he would pull out of the us open just two days before the tournament began—too late however to promote nederer to the no2 seeding', 'category': 'sports'},
         {'paragraph': 'then came the oh-so-familiar djokovic-nadal no-quarter-given battle for dominance in the third set there were exhilarating rallies with both chasing to the net both retrieving what looked like winning shots nadal more than once pulled off a reverse smash and had his chance to seal the tie-break but it was djokovic serving at 10-9 who dragged one decisive error from nadal for a two-sets lead', 'category': 'sports'}
]

df = pd.DataFrame(paragraph_dict_list)
df = df[['paragraph', 'category']]

In [4]:
df.tail()

,paragraph,category
15,the perseyside outfit finished in fourth place...,sports
16,liverpool fc will return to premier league act...,sports
17,alisson signed for liverpool fc from as roma t...,sports
18,but the rankings during that run-in to new yor...,sports
19,then came the oh-so-familiar djokovic-nadal no...,sports


In [5]:
df.head()

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food


**데이터 전처리**

지문에 사용된 모든 단어들을 모아 중복 제거 후, 단어리스트를 만들어 입력값을 수치값으로 변경한다.

In [6]:
# 데이터에 사용된 중복 없는 전체 단어 개수를 리턴합니다.
# set() 의 특성상 동일한 값은 한 번만 저장한다.
def get_vocab_size(df):
  results = set()
  df['paragraph'].str.lower().str.split().apply(results.update)
  return len(results)

# 전체 단어 개수
vocab_size = get_vocab_size(df)
# 단어를 숫자로 인코딩합니다.
paragraphs = df['paragraph'].to_list()
encoded_paragraphs = [one_hot(paragraph, vocab_size) for paragraph in paragraphs]

LSTM의 입력값의 벡터 크기는 동일해야한다.

따라서 가장 긴 문장의 길이를 벡터의 크기로 설정한다.

In [7]:
def get_max_length(df):
  max_length = 0
  for row in df['paragraph']:
    if len(row.split(" ")) > max_length:
      max_length = len(row.split(" "))
    return max_length

max_length = get_max_length(df)
print(max_length)

53


In [8]:
# 문장마다 길이가 다르므로 제로 패딩을 넣는다.
padded_paragraphs_encoding = pad_sequences(encoded_paragraphs, maxlen = max_length,
                                           padding='post')
padded_paragraphs_encoding

array([[341, 236, 451, ..., 212, 424,  17],
       [406,  21, 155, ...,   0,   0,   0],
       [471,  67, 364, ...,   0,   0,   0],
       ...,
       [ 59,  66,  23, ...,   0,   0,   0],
       [100, 295, 355, ..., 321, 477, 406],
       [383, 371, 473, ...,  41,  77, 387]], dtype=int32)

In [9]:
# 레이블 인코딩
categories = df['category'].to_list()
def category_encode(category):
  # 분류항목도 수치로 변경합니다.
  if category == 'food':
    return [1,0]
  else:
    return [0,1]

encoded_category = [category_encode(category) for category in categories]

In [10]:
# 원 핫 인코딩 예시
print(encoded_category[0])
print(encoded_category[18])

[1, 0]
[0, 1]


In [12]:
# 텐서플로 모델 구현
model = Sequential()

# 문맥 벡터 생성 단계
model.add(Embedding(vocab_size, 5, input_length=max_length))
model.add(LSTM(64))

# 분류 단계
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [13]:
train_X = np.array(padded_paragraphs_encoding)
train_Y = np.array(encoded_category)

In [14]:
print("Train. . .")
model.fit(train_X, train_Y, batch_size=10, epochs=50)

Train. . .
Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.4667 - loss: 0.6936
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5667 - loss: 0.6927 
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8000 - loss: 0.6921
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7000 - loss: 0.6914
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6333 - loss: 0.6902
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5667 - loss: 0.6886
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.7333 - loss: 0.6895
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8667 - loss: 0.6847 
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8333 - loss: 0.6817
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8000 - loss: 0.6817
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8333 - loss: 0.6748 
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8333 - 

In [15]:
score, acc = model.evaluate(train_X, train_Y, verbose=2)
print("Test score: ", score)
print("test accuracy: ", acc)

1/1 - 0s - 486ms/step - accuracy: 1.0000 - loss: 6.3317e-04
Test score:  0.0006331719341687858
test accuracy:  1.0


In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (10, 53, 5)                 │           2,680 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (10, 64)                    │          17,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (10, 32)                    │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (10, 2)                     │              66 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 68,240 (266.57 KB)

 Trainable params: 22,746 (88.85 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 45,494 (177.71 KB)